#### iLQR for point mass example with obstacle in 3d

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
from costs import *
from ocp_utils import *
from ocp_sys import *

import pybullet as p
import pybullet_data

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)

## Obstacle Cost Definition 

In [2]:
class ActivationCollision(crocoddyl.ActivationModelAbstract):
    def __init__(self, nr, threshold=0.3):
        crocoddyl.ActivationModelAbstract.__init__(self, nr)
        self.threshold = threshold
        
    def calc(self, r):
        self.d = np.linalg.norm(r)

        if self.d < self.threshold:
            self.a = 0.5*(self.d-self.threshold)**2
        else:
            self.a = 0

    def calcDiff(self, r, recalc=True, use_true_Hessian = True):
        if recalc:
            self.calc(r)
        
        if self.d < self.threshold:
            self.Ar = (self.d-self.threshold)*r/self.d
            if use_true_Hessian:
                self.Arr = (np.eye(self.nr))*(self.d-self.threshold)/self.d + self.threshold*np.outer(r,r)/(self.d**3)
            else:
                self.Arr = np.outer(r,r)/(self.d**2)
                
        else:
            self.Ar = np.zeros(self.nr)
            self.Arr = np.zeros((self.nr,self.nr))

In [3]:
class CostModelCollisionSphereTest():
    '''
    The collision cost model between the end-effector and a sphere obstacle
    '''
    def __init__(self, sys, activation, p_obs, r_obs, ee_id=0, w_obs = 1., d_margin = 0., use_true_Hessian = True):
        self.activation = activation
        self.sys = sys
        self.Dx, self.Du = sys.Dx, sys.Du
        self.dof = self.Du
        self.p_obs = p_obs #obstacle position
        self.r_obs = r_obs #obstacle radius
        
        self.w_obs = w_obs
        self.d_thres = r_obs + d_margin
        self.d_margin = d_margin
        self.obs_status = False
        self.ee_id = ee_id
        self.use_true_Hessian = use_true_Hessian
        
    def calc(self, x, u):
        p,_ = self.sys.compute_ee(x, self.ee_id)
        self.res = p - self.p_obs
        
        self.activation.calc(self.res)
        self.L =self.w_obs*self.activation.a
        return self.L
    
    def calcDiff(self, x, u, recalc = True):
        if recalc:
            self.calc(x, u)
        self.J   = self.sys.compute_Jacobian(x, self.ee_id)[:3]  #Only use the translation part
        p,_      = self.sys.compute_ee(x, self.ee_id)
        
        self.activation.calcDiff( self.res, recalc, self.use_true_Hessian)
        
        self.Lx = np.zeros(self.Dx)
        self.Lx[:self.dof]  = self.w_obs*self.J.T.dot(self.activation.Ar)
        self.Lxx = np.zeros((self.Dx, self.Dx))
        self.Lxx[:self.dof, :self.dof] = self.w_obs*self.J.T.dot(self.activation.Arr).dot(self.J)                

        self.Lu  = np.zeros(self.Du)
        self.Lxu = np.zeros((self.Dx, self.Du))
        self.Luu  = np.zeros((self.Du, self.Du))

## Setup Pybullet

In [4]:
def plot_traj(xs, obj_id, dt = 0.01):
    for x in xs:
        p.resetBasePositionAndOrientation(obj_id, x[:3], (0,0,0,1))
        time.sleep(dt)

In [5]:
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [6]:
p.resetSimulation()

#p.loadURDF('plane.urdf')
_,_,obj_id = create_primitives(rgbaColor=[0,0,1,1],radius = 0.04)
_,_,target_id = create_primitives(radius = 0.04, rgbaColor=[0,1,0,0.5])

## Setup the System

#### Create point mass system with dimension 3 (double integrator)

In [7]:
dt = 0.05  #duration of 1 time step
T = 100    #total number of time steps
Dx, Du = 6, 3 #dimensions of x and u
dof = 3

#Define the matrix A and B to define a double integrator
A = np.eye(Dx)
A[:dof,dof:] = np.eye(dof)*dt
B = np.zeros((Dx, Du))
B[:dof,:] = 0.5*np.eye(Du)*(dt**2)
B[dof:, :] = np.eye(Du)*dt

sys = LinearSystem(A, B)

#### Set initial state and control trajectory

In [8]:
x0 = np.ones(Dx)*-1
x0[3:] *=0
sys.set_init_state(x0)

#set initial control output to be all zeros
us_init = np.zeros((T+1,sys.Du))
_ = sys.compute_matrices(x0, us_init[0])
xs_init = sys.rollout(us_init[:-1])

#### Plot initial trajectory

#### Set the regularization cost coefficients Q and R 

In [9]:
Q = np.eye(sys.Dx)*0.01 #coefficient for running cost
Qf = np.eye(sys.Dx)*10  #coefficient for terminal cost
R = np.eye(sys.Du)*0.005  #control coefficient
mu = 1e-6              #regularization coefficient

#### Set reference target

In [10]:
x_target = np.array([1, 1, 1, 0,0, 0])

#### Set obstacle

In [11]:
p_obs1 = np.array([0.01, 0., 0.])   #obstacle position
r_obs1 = 0.35 #radius of the obstacle
w_obs1 = 100.   #weight of the obstacle cost
d_margin1 = 0.1 #margin of the obstacle

#### Show the setting in pybullet

In [12]:
_,_,obs_id = create_primitives(radius = r_obs1)
p.resetBasePositionAndOrientation(obs_id, p_obs1, (0,0,0,1))
p.resetBasePositionAndOrientation(obj_id, x0[:3], (0,0,0,1))
p.resetBasePositionAndOrientation(target_id, x_target[:3], (0,0,0,1))

## Setup the ILQR & Solve

#### Define the cost

In [13]:
cost_names_dict = {'state':0, 'control':1, 'obstacle':2}

#The costs consist of: a) state tracking (Q), b) control regularization (R), c) obstacle cost
#Running cost is for the time 0 <= t < T, while terminal cost is for the time t = T
costs = []
for i in range(T):
    runningStateCost = CostModelQuadratic(sys, Q, x_ref = x_target)
    runningControlCost = CostModelQuadratic(sys, None, R)
    activationObstacle = ActivationCollision(3, threshold=r_obs1 + d_margin1)
    runningObstacleCost = CostModelCollisionSphereTest(sys, activationObstacle, p_obs1, r_obs1, w_obs = w_obs1, d_margin=d_margin1, use_true_Hessian=True)    
    runningCost = CostModelSum(sys, [runningStateCost, runningControlCost, runningObstacleCost])
    costs += [runningCost]

terminalStateCost = CostModelQuadratic(sys,Qf, x_ref = x_target)
terminalControlCost = CostModelQuadratic(sys, None,R)
activationObstacle = ActivationCollision(3, threshold=r_obs1 + d_margin1)
terminalObstacleCost = CostModelCollisionSphereTest(sys, activationObstacle, p_obs1, r_obs1, w_obs = w_obs1, d_margin=d_margin1, use_true_Hessian=True)    
terminalCost = CostModelSum(sys, [terminalStateCost, terminalControlCost, terminalObstacleCost])

costs += [terminalCost]

#### Setup crocoddyl model

In [14]:
#### Setup crocoddyl model
rmodels = []
for i in range(T):
    state = crocoddyl.StateVector(sys.Dx)
    rmodel = ActionModelRobot(state, sys.Du)
    rmodel.init_robot_sys(sys, nr = sys.Dx)
    rmodel.set_cost(costs[i])
    rmodels += [rmodel]

rmodel_T = ActionModelRobot(state, sys.Du)
rmodel_T.init_robot_sys(sys, nr = sys.Dx)
rmodel_T.set_cost(terminalCost)

problem = crocoddyl.ShootingProblem(x0, rmodels, rmodel_T)
ddp = crocoddyl.SolverFDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

#### Solve ilqr with True Hessian

In [15]:
for i in range(T):
    ddp.problem.runningModels[i].cost_model.costs[cost_names_dict['obstacle']].use_true_Hessian = True
ddp.problem.terminalModel.cost_model.costs[cost_names_dict['obstacle']].use_true_Hessian = True
    

ddp.solve(list(xs_init[:,:,None]), list(us_init[:-1,:,None]), maxiter=100)
xs, us = np.array(ddp.xs), np.array(ddp.us)

#### Solve ilqr with Gauss Newton

In [16]:
for i in range(T):
    ddp.problem.runningModels[i].cost_model.costs[cost_names_dict['obstacle']].use_true_Hessian = False
ddp.problem.terminalModel.cost_model.costs[cost_names_dict['obstacle']].use_true_Hessian = False
    

ddp.solve(list(xs_init[:,:,None]), list(us_init[:-1,:,None]), maxiter=100)
xs, us = np.array(ddp.xs), np.array(ddp.us)

In [192]:
plot_traj(xs[:100], obj_id, dt=0.1)

***